# 解答例

この演習では，日本の歌謡曲の歌詞データを対象に分析を行う．

使用するデータは，山本がオリコンランキングから収集した歌謡曲の歌詞に関するものである．
[オリコンランキング](https://www.oricon.co.jp/rank/)は，日本で最も知名度のある音楽ヒットランキングである．
オリコンランキングの一つに[年間シングル売り上げランキング](https://www.oricon.co.jp/rank/js/y/2024/)というものがあり，ある年のシングル曲売り上げを集計し，そのランキングを1968年から公開し続けている．

[こちらのリンク](https://www.ds.nagoya-cu.ac.jp/~yamamoto/download/jsong-lyrics/data.zip)からダウンロードできるファイル`data.zip`内には，ある年度のシングル曲売り上げ上位30件の曲の歌詞の情報が記された[TSVファイル](https://ja.wikipedia.org/wiki/Tab-Separated_Values)が1968年から2024年分まで納められている（歌詞提供サイトの都合上，一部の曲が抜け落ちている）．
TSVファイル名は`{順位}_{アーティスト名}_{曲名}.tsv`の形式となっている．
例えば，ダウンロードした`data.zip`ファイルを解凍して得られる`data/tokenized/2024`年のフォルダに格納された`01_Snow Man_LOVE TRIGGER.tsv`というファイルは，「2024年のシングル売り上げ1位はSnow ManがリリースしたLOVE TRIGGERという曲」についての歌詞情報が記録されている．

また，各TSVファイルには，ファイル名に関する曲の歌詞を品詞単位で分割し，分割された単語に関する情報が一行ごとに格納されている．
例えば，以下は`data/tokenized/2018/`フォルダにある`18_米津玄師_Lemon.tsv`ファイルの冒頭10行である．

```
夢	夢	名詞	一般
なら	だ	助動詞	*
ば	ば	助詞	接続助詞
どれほど	どれほど	副詞	一般
よかっ	よい	形容詞	自立
た	た	助動詞	*
でしょ	です	助動詞	*
う	う	助動詞	*
未だに	未だに	副詞	一般
あなた	あなた	名詞	代名詞

```

行は歌詞に出現した単語に対応しており，出現順に並んでいる．
各行のテキストは（見えないが）タブ文字（\t）で区切られており，左から順に
- 活用形（歌詞中で実際に使われた形）
- 原形（辞書に載る基本形）
- 品詞（名詞・動詞など）
- 品詞詳細（助詞・代名詞などの細分類）

に関する情報が入っている．
例えば，上の例では5行目の情報からは
- 米津玄師作のLemonの歌詞で5番目に出現する単語は「よかっ」で
- 品詞は「形容詞」，原形は「よい」である

ということが読み取れる．

## 準備
以下のコードを実行すると，演習で用いるファイルをダウンロードし，`data/tokenized`フォルダにデータを展開できる．
なお，ダウンロードしたzipファイルを解答するにはパスワードが必要となる．
**教員からパスワードを入手すること**．

1. `data.zip`ファイルをまだダウンロードしていない人は，以下のコードを実行してください．
2. `data.zip`ファイルを既にダウンロードした人，あるいはファイルを自力でダウンロードする人は，以下の手順をスキップしてください．

In [1]:
import os
import requests
import zipfile

# ダウンロードURL
DATA_URL = 'https://www.ds.nagoya-cu.ac.jp/~yamamoto/download/jsong-lyrics/data.zip'

# パスワード
ZIP_PASSWORD = '******'

# データをダウンロード（要学内ネットワーク）
print("[STEP] ダウンロード開始（学内のみ）...")
downloaded_data = requests.get(DATA_URL).content
with open(f'data.zip', mode='wb') as f: # wb でバイト型を書き込める
  f.write(downloaded_data)

# Zipファイル解凍
print("[STEP] 解凍中...")
zip_data = zipfile.ZipFile('data.zip', 'r')
zip_data.extractall(path='.', pwd=ZIP_PASSWORD.encode())
zip_data.close()

# ゴミファイルの削除
print("[STEP] 余分ファイル削除中...")
for year in range(1968, 2025):
    for filename in os.listdir(f'data/tokenized/{year}'):
        if not filename.endswith('.tsv'):
            os.remove(f'data/tokenized/{year}/{filename}')

# ダウンロードしたzipファイルを削除
os.remove('data.zip')
print("[DONE] data/tokenized/ にTSVが展開されました。")

[STEP] ダウンロード開始（学内のみ）...
[STEP] 解凍中...
[STEP] 余分ファイル削除中...
[DONE] data/tokenized/ にTSVが展開されました。


## 演習の進め方

上記を踏まえて，以下の課題に取り組みなさい．

課題コンテンツは以下の2種類の要素で構成されている：
- 提出課題：実データを扱う本番問題（採点対象），全部で5つ
- 練習問題：提出課題を解くための基礎練習
  
練習を飛ばして提出課題に挑戦しても構いません．
提出課題が難しいと感じる場合は，練習問題から順に進めるのが効果的です．

---

## 課題41

#### 練習41-A: 文字列を区切る

学生に読んだ本を毎日記録し1週間ごとに報告するよう依頼した．
依頼の際，データ分析の都合上，読んだ本をカンマ（,）で区切って書くようお願いした．
その結果，学生Xから先週読んだ本の情報について以下のような内容が送られてきた：

```
コンビニ人間,ノルウェイの森,変身,ノルウェイの森,夜は短し歩けよ乙女,コンビニ人間,ノルウェイの森
```

今，上記内容を以下のように変数`text`に格納したとしよう．
`text`の内容をカンマで区切り，本の名前を要素とするリスト`books`を作成するコードを書きなさい．

```
text = "コンビニ人間,ノルウェイの森,変身,ノルウェイの森,夜は短し歩けよ乙女,コンビニ人間,ノルウェイの森"
```

##### 関連する事項: [str.split](https://note.nkmk.me/python-split-rsplit-splitlines-re/)

In [2]:
text = "コンビニ人間,ノルウェイの森,変身,ノルウェイの森,夜は短し歩けよ乙女,コンビニ人間,ノルウェイの森"

books = text.split(',')
print(books)

['コンビニ人間', 'ノルウェイの森', '変身', 'ノルウェイの森', '夜は短し歩けよ乙女', 'コンビニ人間', 'ノルウェイの森']


#### 練習41-B: 辞書データの作成（リストのループ/setdefault）

以下のリストは，学生Yが1週間に読んだ本のタイトルである．
それぞれの本が何回読まれたかの情報について，変数`book_frequency`に辞書型データとして格納しなさい．

```
books = ["ハリーポッター", "ハリーポッター", "進撃の巨人", "ワンピース", "進撃の巨人", "ワンピース", "ワンピース"]
```

##### 関連する事項: [リストのループ](https://qiita.com/motoki1990/items/d06fc7559546a8471392), [dict.setdefault](https://note.nkmk.me/python-dict-setdefault/)

In [3]:
books = ["ハリーポッター", "ハリーポッター", "進撃の巨人", "ワンピース", "進撃の巨人", "ワンピース", "ワンピース"]

book_frequency = {}
for book in books:
    book_frequency.setdefault(book, 0)
    book_frequency[book] += 1

print(book_frequency)

{'ハリーポッター': 2, '進撃の巨人': 2, 'ワンピース': 3}


#### 練習41-C: 辞書のソート

練習41-Cで作成した辞書データを用いて，学生Yが読んだ本のタイトルと読書回数を回数が多い順に表示するコードを書きなさい．

##### 関連する事項: [sorted](https://note.nkmk.me/python-dict-list-sort/)


In [4]:
sorted_books = sorted(book_frequency.items(), key=lambda x: x[1], reverse=True)
for book, frequency in sorted_books:
    print(book, frequency)


ワンピース 3
ハリーポッター 2
進撃の巨人 2


### ☆課題41（提出課題）
2019年のシングル曲売り上げ上位30件（のファイル名）を調べ，ランキング30位以内に入った回数が最も多いアーティストを表示するコードを書きなさい．

##### 関連する事項: [os.listdir](https://note.nkmk.me/python-listdir-isfile-isdir/)


##### ヒント
指定したフォルダ（ディレクトリ）の中にあるファイルの一覧を取得するには，`os`モジュールの`listdir`関数を使うとよい．
以下は，`data/tokenized/2024`という場所にあるフォルダ内にあるファイル一覧を取得するコードの例である．

In [5]:
import os

dir_path = 'data/tokenized/2024'
filenames = os.listdir(dir_path)

print(filenames)

['22_なにわ男子_コイスルヒカリ.tsv', '13_櫻坂46_I want tomorrow to come.tsv', '19_AKB48_恋 詰んじゃった.tsv', '01_Snow Man_LOVE TRIGGER.tsv', '20_AKB48_カラコンウインク.tsv', '04_Aぇ! group_《A》BEGINNING.tsv', '24_ZEROBASEONE_ゆらゆら -運命の花-.tsv', '23_Aぇ! group_Gotta Be.tsv', '08_JO1_WHERE DO WE GO.tsv', '18_SixTONES_GONG.tsv', '15_日向坂46_君はハニーデュー.tsv', '07_乃木坂46_チャンスは平等.tsv', '14_日向坂46_絶対的第六感.tsv', '21_SKE48_愛のホログラム.tsv', '10_SixTONES_音色.tsv', '17_SEVENTEEN_消費期限.tsv', '09_乃木坂46_チートデイ.tsv', '29_WEST_ハート.tsv', '28_&TEAM_五月雨(Samidare).tsv', '26_RIIZE_Lucky.tsv', '02_Snow Man_BREAKOUT.tsv', '27_King & Prince_halfmoon.tsv', '16_櫻坂46_何歳の頃に戻りたいのか.tsv', '05_櫻坂46_自業自得.tsv', '30_Sexy Zone_puzzle.tsv']


In [6]:
import os

dir_path = 'data/tokenized/2019'

frequency_artist = {}

filenames = os.listdir(dir_path)
for filename in filenames:
    rank, artist, title = filename.split('_')
    title = title.replace('.tsv', '')
    frequency_artist.setdefault(artist, 0)
    frequency_artist[artist] += 1

sorted_frequency_artist = sorted(frequency_artist.items(), key=lambda x: x[1], reverse=True)
top_artist, freq = sorted_frequency_artist[0]
print('1位', top_artist, f'(ランクイン回数: {freq})')

1位 日向坂46 (ランクイン回数: 3)


---

## 課題42

#### 練習42-A: リスト/辞書のループ

以下のデータは，いくつかの架空のスポーツ大会において優勝した大学の名前を年ごとにまとめたものである．
ご覧の通り，変数`results`に格納されたデータは（Pythonの）辞書型データである．


```
results = {
    "2021": ["A大学", "B大学", "A大学", "C大学"],
    "2022": ["B大学", "B大学", "C大学", "A大学"],
    "2023": ["A大学", "C大学", "C大学", "D大学"]
}
```

変数`results`に格納されたデータを用いて，キーを「大学名」，値を「優勝回数」とする辞書型データ`frequency_win`を作成するコードを書きなさい．

##### 関連する事項: [辞書のループ](https://note.nkmk.me/python-dict-keys-values-items/), [dict.setdefault](https://note.nkmk.me/python-dict-setdefault/)

In [7]:
results = {
    "2021": ["A大学", "B大学", "A大学", "C大学"],
    "2022": ["B大学", "B大学", "C大学", "A大学"],
    "2023": ["A大学", "C大学", "C大学", "D大学"]
}

frequency_win = {}

for year in results:
    for university in results[year]:
        frequency_win.setdefault(university, 0)
        frequency_win[university] += 1

print(frequency_win)


{'A大学': 4, 'B大学': 3, 'C大学': 4, 'D大学': 1}


#### 練習42-B: 辞書のソート

練習42-Aで変数`frequency_win`に格納したデータを用いて，優勝回数の下位3大学を表示するコードを書きなさい．

##### 関連する事項: [sorted](https://note.nkmk.me/python-dict-list-sort/), [スライスによるリストのアクセス](https://note.nkmk.me/python-slice-usage/)

In [8]:
sorted_universities = sorted(frequency_win.items(), key=lambda x: x[1])

for university, win_count in sorted_universities[:3]:
    print(university, win_count)


D大学 1
B大学 3
A大学 4


### ☆課題42（提出課題）
1968年から2024年までのシングル曲売り上げ上位30件（のファイル名）を調べ，ランキング30位以内に入った回数が多いアーティストの上位10件を表示するコードを書きなさい．

##### 関連する事項: [range](https://note.nkmk.me/python-range-usage/), [os.listdir](https://note.nkmk.me/python-listdir-isfile-isdir/), [str.split](https://note.nkmk.me/python-split-rsplit-splitlines-re/)

In [9]:
import os

dir_path = 'data/tokenized/'

frequency_artist = {}

for year in range(1968, 2025):
    target_dir_path = f'{dir_path}/{year}'
    filenames = os.listdir(target_dir_path)
    for filename in filenames:
        rank, artist, title = filename.split('_')
        title = title.replace('.tsv', '')
        frequency_artist.setdefault(artist, 0)
        frequency_artist[artist] += 1

sorted_frequency_artist = sorted(frequency_artist.items(), key=lambda x: x[1], reverse=True)

for rank in range(10):
    top_artist, freq = sorted_frequency_artist[rank]
    print(f'{rank+1}位', top_artist, f'(ランクイン回数: {freq})')

1位 AKB48 (ランクイン回数: 49)
2位 嵐 (ランクイン回数: 38)
3位 乃木坂46 (ランクイン回数: 36)
4位 B'z (ランクイン回数: 34)
5位 SKE48 (ランクイン回数: 27)
6位 KinKi Kids (ランクイン回数: 23)
7位 NMB48 (ランクイン回数: 20)
8位 浜崎あゆみ (ランクイン回数: 19)
9位 松田聖子 (ランクイン回数: 18)
10位 中森明菜 (ランクイン回数: 18)


---

## 課題43

#### 練習43-A: ファイルの読み込み

[こちら](data/fruit.txt)にあるファイル（`fruit.txt`）には次のような内容が書かれている：

```
りんご,440
バナナ,220
みかん,330
...
```

`fruit.txt`ファイルをダウンロードした上で，ファイルの内容を1行ずつ読み込み，果物名をキー，数字（価格）を値とする辞書データを作成しなさい．
作成したデータは変数`fruits`に格納しなさい（※ ダウンロードは手動で行えばよい）．

##### 関連する事項: [ファイルの読み込み](https://note.nkmk.me/python-file-io-open-with/), [str.split](https://note.nkmk.me/python-split-rsplit-splitlines-re/)

In [10]:
fruits = {}

with open('data/fruit.txt') as f:
    for line in f:
        clean_line = line.strip()
        fruit, price = clean_line.split(',')
        fruits[fruit] = int(price)

print(fruits)

{'りんご': 440, 'バナナ': 220, 'みかん': 330, 'いちご': 1400, 'メロン': 1050, 'すいか': 350}


#### 練習43-B: 辞書のキーの一覧/文字列結合

練習43-Aで作成した辞書`fruit`について，その辞書のキー（つまり果物名）をカンマ（,）で結合し，文字列として出力するコードを書きなさい．

##### 関連する事項: [dict.keys](https://note.nkmk.me/python-dict-keys-values-items/), [str.join](https://note.nkmk.me/python-string-concat/)

In [11]:
fruit_names = list(fruits.keys())

print(",".join(fruit_names))

りんご,バナナ,みかん,いちご,メロン,すいか


### ☆提出課題43

`data/tokenized/1976`フォルダにある`01_子門真人_およげ!たいやきくん.tsv`ファイルを1行ずつ読み込み，単語の出現順に「活用形」を取り出しつなげることで，「およげ!たいやきくん」の歌詞を復元しなさい．


##### 関連する事項: [ファイルの読み込み](https://note.nkmk.me/python-file-io-open-with/), [str.join](https://note.nkmk.me/python-string-concat/)

In [12]:
filepath = 'data/tokenized/1976/01_子門真人_およげ!たいやきくん.tsv'

lyrics = ''
with open(filepath) as f:
    for line in f:
        term_data = line.split('\t')
        surface = term_data[0] # 1つ目の情報が活用形
        lyrics = lyrics + surface

print(lyrics)


毎日毎日　僕らは鉄板の　上で焼かれて　嫌になっちゃうよある朝　僕は　店のおじさんと　けんかして　海に逃げこんだのさ初めて泳いだ海の底　とっても気持ちがいいもんだお腹のあんこが重いけど　海は広いぜ心がはずむ桃いろさんごが手を振って　僕の泳ぎを眺めていたよ毎日毎日　楽しいことばかり　難破船が僕のすみかさときどき鮫に　いじめられるけど　そんなときゃ　そうさ　逃げるのさ一日泳げばはらぺこさ　目玉もくるくる回っちゃうたまにはえびでも食わなけりゃ　塩水ばかりじゃふやけてしまう岩場のかげから食いつけば　それは小さなつりばりだったどんなにどんなに　もがいても　針が喉からとれないよ浜べで見知らぬおじさんが　僕を釣り上げびっくりしてたやっぱり僕はたいやきさ　少し焦げあるたいやきさおじさん　つばをのみ込んで　僕をうまそに食べたのさ


---

## 課題44

#### 練習44-A: 辞書のループ/条件分岐

変数`animals`は，動物とその分類に関する情報が格納された辞書である．

```
animals = {
    "犬": "哺乳類",
    "カエル": "両生類",
    "トカゲ": "爬虫類",
    "猫": "哺乳類",
    "ワニ": "爬虫類"
}
```

分類が哺乳類の動物だけを抽出し，その名前（動物名）を要素とするリスト`animal_names`を作成するコードを書きなさい．

##### 関連する事項: [dict.items](https://note.nkmk.me/python-dict-keys-values-items/)

In [13]:
animals = {
    "犬": "哺乳類",
    "カエル": "両生類",
    "トカゲ": "爬虫類",
    "猫": "哺乳類",
    "ワニ": "爬虫類"
}

animal_names = []

for animal_name, kind in animals.items():
    if kind == '哺乳類':
        animal_names.append(animal_name)

print(animal_names)

['犬', '猫']


#### 練習44-B: 条件つきカウント

練習44-Aで用いた変数`animals`において，分類が「哺乳類」以外の動物の数をカウントするコードを書きなさい．

##### 関連する事項: [dict.items](https://note.nkmk.me/python-dict-keys-values-items/)

In [14]:
animals = {
    "犬": "哺乳類",
    "カエル": "両生類",
    "トカゲ": "爬虫類",
    "猫": "哺乳類",
    "ワニ": "爬虫類"
}

# Write your code below
non_mammals = []

for animal, kind in animals.items():
    if kind != '哺乳類':
        non_mammals.append(animal)

num_non_mammal = len(non_mammals)
print(num_non_mammal)

3


### ☆提出課題44

歌詞における代名詞の使用頻度を分析したい．
1968年から2024年までのランキングに入っている曲の歌詞から代名詞を抽出し，その代名詞が歌詞中で用いられている曲数を辞書型で格納しなさい（代名詞をキー，曲数を値とする辞書データを作成しなさい）．
また，使用頻度（使用されている曲数）が高い順に代名詞をソートし，その上位20件を表示しなさい．

※ 代名詞を抽出する際，以下の指示語は無視すること．

```
STOPWORDS = ['これ', 'それ', 'あれ', 'どれ', 'ここ', 'そこ', 'あそこ', 'どこ']
```

##### 関連する事項: [range](https://note.nkmk.me/python-range-usage/), [os.listdir](https://note.nkmk.me/python-listdir-isfile-isdir/), [ファイルの読み込み](https://note.nkmk.me/python-file-io-open-with/), [dict.items](https://note.nkmk.me/python-dict-keys-values-items/), [dict.setdefault](https://note.nkmk.me/python-dict-setdefault/)

In [15]:
import os

STOPWORDS = ['これ', 'それ', 'あれ', 'どれ', 'ここ', 'そこ', 'あそこ', 'どこ']

top_k = 20
data_dir = 'data/tokenized'
lyrics_frequency_pronoun = {}

for year in range(1968, 2025):
    target_data_dir = f'{data_dir}/{year}'

    filenames = os.listdir(target_data_dir)
    for filename in filenames:
        with open(target_data_dir + '/' + filename) as f:
            pronoun_frequency = {}

            for line in f:
                surface, baseform, pos, pos_detail = line.rstrip().split('\t')

                if pos_detail == '代名詞' and surface not in STOPWORDS:
                    pronoun_frequency.setdefault(surface, 0)
                    pronoun_frequency[surface] += 1

        for pronoun in pronoun_frequency.keys():
            lyrics_frequency_pronoun.setdefault(pronoun, 0)
            lyrics_frequency_pronoun[pronoun] += 1


ranked_pronouns = sorted(lyrics_frequency_pronoun.items(), key=lambda x: x[1], reverse=True)
for pronoun, freq in ranked_pronouns[:top_k]:
    print(f"{pronoun} - {freq}回")

君 - 596回
誰 - 493回
僕 - 433回
あなた - 430回
何 - 413回
いつ - 312回
私 - 307回
僕ら - 156回
俺 - 134回
みんな - 129回
おまえ - 52回
お前 - 50回
わたし - 49回
何処 - 49回
ぼく - 49回
いくつ - 47回
なん - 41回
彼 - 37回
キミ - 37回
あいつ - 32回


---

## 課題45

#### 練習45-A: 辞書の辞書/ループの入れ子

変数`sales`に格納されたデータは，年代ごとに商品A, B, Cが売れた個数を示している．
各商品の合計販売数を求めるコードを書きなさい（例: Aは18個）．

```
sales = {
    1990: {"A": 10, "B": 5},
    2000: {"A": 3, "C": 8},
    2010: {"B": 7, "C": 10},
    2020: {"A": 5, "B": 2, "C": 3}
}
```

##### 関連する事項: [dict.items](https://note.nkmk.me/python-dict-keys-values-items/), [dict.setdefault](https://note.nkmk.me/python-dict-setdefault/)

In [16]:
sales = {
    1990: {"A": 10, "B": 5},
    2000: {"A": 3, "C": 8},
    2010: {"B": 7, "C": 10},
    2020: {"A": 5, "B": 2, "C": 3}
}

# Write your code below
sale_by_product = {}

for year in sales:
    for product, sale in sales[year].items():
        sale_by_product.setdefault(product, 0)
        sale_by_product[product] += sale

print(sale_by_product)

{'A': 18, 'B': 14, 'C': 21}


#### 練習45-B: 辞書ソート再び

練習45-Aで用いたデータにおいて，最も販売合計数が多い商品名とその数を出力するコードを書きなさい．

##### 関連する事項: [dict.items](https://note.nkmk.me/python-dict-keys-values-items/), [dict.setdefault](https://note.nkmk.me/python-dict-setdefault/), [sorted](https://note.nkmk.me/python-dict-list-sort/)

In [17]:
sorted_products = sorted(sale_by_product.items(), key=lambda x: x[1], reverse=True)
top_product = sorted_products[0]
print(top_product)

('C', 21)


### ☆提出課題45

歌詞における言葉の使われ方は，社会や文化，時代ごとの流行を反映していると考えられる．
そこでこの課題では，「僕」「君」といった一人称/二人称代名詞に着目したい．

以下の`PRONOUNS`に記した一人称/二人称代名詞について，注目する代名詞が出現する曲数を年代別に分析し，結果を表示しなさい．
なお，「年代」のグルーピングは1960年代，1970年代，1980年代，1990年代，2000年代，2010年代，2020年代としなさい．

```
PRONOUNS = ["僕", "私", "僕ら", "俺", "わたし", "ぼく", "君", "あなた", "おまえ", "お前", "キミ"]
```

##### 関連する事項: [os.listdir](https://note.nkmk.me/python-listdir-isfile-isdir/), [ファイルの読み込み](https://note.nkmk.me/python-file-io-open-with/), [リストのループ](https://qiita.com/motoki1990/items/d06fc7559546a8471392), [dict.items/keys](https://note.nkmk.me/python-dict-keys-values-items/), [dict.setdefault](https://note.nkmk.me/python-dict-setdefault/), [in](https://note.nkmk.me/python-in-basic/), [sorted](https://note.nkmk.me/python-dict-list-sort/)

In [18]:
import os

PRONOUNS = ["僕", "私", "僕ら", "俺", "わたし", "ぼく", "君", "あなた", "おまえ", "お前", "キミ"]

data_dir = 'data/tokenized'
lyrics_frequency_pronoun = {}

for year in range(1968, 2025):
    age = int(year / 10) * 10

    target_data_dir = f'{data_dir}/{year}'

    filenames = os.listdir(target_data_dir)
    for filename in filenames:
        with open(target_data_dir + '/' + filename) as f:
            pronoun_frequency = {}

            for line in f:
                surface, baseform, pos, pos_detail = line.rstrip().split('\t')

                if surface in PRONOUNS:
                    pronoun_frequency.setdefault(surface, 0)
                    pronoun_frequency[surface] += 1

            for pronoun in pronoun_frequency.keys():
                lyrics_frequency_pronoun.setdefault(age, {})
                lyrics_frequency_pronoun[age].setdefault(pronoun, 0)
                lyrics_frequency_pronoun[age][pronoun] += 1


for year, noun_frequency in lyrics_frequency_pronoun.items():
    print("\n=============")
    print(f"{year}年代")
    print("=============")
    for noun, freq in noun_frequency.items():
        print(f"{noun} - {freq}回")




1960年代
おまえ - 1回
あなた - 19回
君 - 10回
私 - 20回
僕 - 5回
俺 - 3回
お前 - 2回
わたし - 3回
ぼく - 4回

1970年代
私 - 94回
あなた - 133回
わたし - 24回
おまえ - 10回
ぼく - 19回
俺 - 18回
君 - 46回
僕 - 34回
お前 - 9回
僕ら - 2回

1980年代
私 - 71回
あなた - 97回
君 - 61回
僕 - 28回
おまえ - 20回
僕ら - 1回
俺 - 45回
わたし - 15回
お前 - 23回
ぼく - 7回

1990年代
君 - 107回
私 - 46回
あなた - 82回
ぼく - 14回
僕 - 65回
キミ - 5回
俺 - 18回
お前 - 4回
おまえ - 10回
僕ら - 20回
わたし - 3回

2000年代
君 - 148回
僕 - 109回
俺 - 25回
僕ら - 57回
キミ - 19回
私 - 40回
あなた - 56回
お前 - 5回
ぼく - 5回
わたし - 1回
おまえ - 3回

2010年代
君 - 150回
僕 - 122回
キミ - 10回
あなた - 33回
僕ら - 46回
俺 - 18回
私 - 28回
わたし - 3回
お前 - 4回
おまえ - 6回

2020年代
君 - 86回
僕 - 70回
僕ら - 30回
あなた - 10回
私 - 8回
キミ - 4回
わたし - 2回
おまえ - 2回
俺 - 7回
お前 - 3回
